# Movie Recommender Systems 

A basic recommendation system that suggests movies/items most similiar to movie choice.

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Get the Data

In [2]:
#importing movie metadata
meta= pd.read_csv("movies_metadata.csv")
meta= meta[['id', 'original_title', 'original_language']]
meta= meta.rename(columns={'id':'movieId'})
meta = meta[meta['original_language']== 'en'] #just want movies in English
meta.head()

/Users/kris/anaconda3/anaconda/envs/softwarepark/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,movieId,original_title,original_language
0,862,Toy Story,en
1,8844,Jumanji,en
2,15602,Grumpier Old Men,en
3,31357,Waiting to Exhale,en
4,11862,Father of the Bride Part II,en


In [3]:
#importing movie ratings
ratings= pd.read_csv("ratings_small.csv")
ratings= ratings[['userId', 'movieId', 'rating']]

In [4]:
#convert data types before merging
meta.movieId =pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors= 'coerce')

We can merge them together:

In [5]:
#create a single dataset merging the previous 2
data= pd.merge(ratings, meta, on='movieId', how='inner')
data.head()

,userId,movieId,rating,original_title,original_language
0,1,1371,2.5,Rocky III,en
1,4,1371,4.0,Rocky III,en
2,7,1371,3.0,Rocky III,en
3,19,1371,4.0,Rocky III,en
4,21,1371,3.0,Rocky III,en


In [6]:
#movie matrix so that I can use the recommender function later
matrix= data.pivot_table(index='userId', columns='original_title', values='rating')
matrix.head()

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
matrix.dtypes

original_title
!Women Art Revolution                          bool
'Gator Bait                                    bool
'Twas the Night Before Christmas               bool
10 Items or Less                               bool
10 Things I Hate About You                     bool
10,000 BC                                      bool
11'09''01 - September 11                       bool
12 + 1                                         bool
12 Angry Men                                   bool
1408                                           bool
15 Minutes                                     bool
16 Blocks                                      bool
1984                                           bool
2 Days in Paris                                bool
20,000 Leagues Under the Sea                   bool
2001: A Space Odyssey                          bool
2010                                           bool
2061 - Un anno eccezionale                     bool
21 Grams                                       bo

In [7]:
matrix = matrix.notnull()
matrix.head()

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [8]:
frequent_itemsets = apriori(matrix, min_support=0.07, use_colnames=True)

In [9]:
frequent_itemsets

,support,itemsets
0,0.131148,"(20,000 Leagues Under the Sea)"
1,0.129657,(2001: A Space Odyssey)
2,0.070045,(28 Weeks Later)
3,0.298063,(48 Hrs.)
4,0.298063,(5 Card Stud)
5,0.098361,(A Brief History of Time)
6,0.152012,(A Clockwork Orange)
7,0.268256,(A Nightmare on Elm Street)
8,0.196721,(A River Runs Through It)
9,0.089419,(A Time to Kill)


In [10]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(48 Hrs.),"(20,000 Leagues Under the Sea)",0.298063,0.131148,0.076006,0.255000,1.944375,0.036916,1.166245
1,"(20,000 Leagues Under the Sea)",(48 Hrs.),0.131148,0.298063,0.076006,0.579545,1.944375,0.036916,1.669473
2,(A Nightmare on Elm Street),"(20,000 Leagues Under the Sea)",0.268256,0.131148,0.081967,0.305556,2.329861,0.046786,1.251148
3,"(20,000 Leagues Under the Sea)",(A Nightmare on Elm Street),0.131148,0.268256,0.081967,0.625000,2.329861,0.046786,1.951316
4,(Back to the Future Part II),"(20,000 Leagues Under the Sea)",0.211624,0.131148,0.077496,0.366197,2.792254,0.049742,1.370856


In [11]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
86296,"(Psycho, Live and Let Die)","(Big Fish, Rain Man)",0.086438,0.132638,0.071535,0.827586,6.239442,0.060070,5.030700
86782,"(Night on Earth, Big Fish)","(Psycho, Rain Man)",0.093890,0.131148,0.076006,0.809524,6.172619,0.063693,4.561475
164131,"(Night on Earth, Titanic, Big Fish)","(A Nightmare on Elm Street, Psycho)",0.083458,0.131148,0.070045,0.839286,6.399554,0.059099,5.406193
169445,"(A Nightmare on Elm Street, Lost in Translatio...","(The Conversation, Reservoir Dogs)",0.077496,0.149031,0.070045,0.903846,6.064808,0.058495,8.850075
205923,"(Night on Earth, Titanic, Big Fish)","(Psycho, Rain Man)",0.083458,0.131148,0.071535,0.857143,6.535714,0.060590,6.081967
206375,"(Romeo + Juliet, Psycho, Rain Man)","(Big Fish, Titanic)",0.077496,0.150522,0.070045,0.903846,6.004760,0.058380,8.834575
207393,"(The Conversation, Psycho, To Kill a Mockingbird)","(Big Fish, Silent Hill)",0.086438,0.125186,0.070045,0.810345,6.473112,0.059224,4.612654
236102,"(Titanic, Silent Hill, Syriana)","(Reservoir Dogs, To Kill a Mockingbird)",0.095380,0.138599,0.080477,0.843750,6.087702,0.067257,5.512966
240691,"(Back to the Future Part II, Reservoir Dogs, 4...","(A Nightmare on Elm Street, Titanic, Rain Man)",0.086438,0.135618,0.073025,0.844828,6.229443,0.061303,5.570459
250425,"(Big Fish, Back to the Future Part II, Reservo...","(A Nightmare on Elm Street, Titanic, Rain Man)",0.080477,0.135618,0.070045,0.870370,6.417786,0.059131,6.668086


In [12]:
rules.sum()

antecedent support    4.952688e+04
consequent support    4.952688e+04
support               2.211741e+04
confidence            1.514094e+05
lift                  9.217685e+05
leverage              1.460901e+04
conviction                     inf
dtype: float64

In [13]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(48 Hrs.),"(20,000 Leagues Under the Sea)",0.298063,0.131148,0.076006,0.255000,1.944375,0.036916,1.166245,1
1,"(20,000 Leagues Under the Sea)",(48 Hrs.),0.131148,0.298063,0.076006,0.579545,1.944375,0.036916,1.669473,1
2,(A Nightmare on Elm Street),"(20,000 Leagues Under the Sea)",0.268256,0.131148,0.081967,0.305556,2.329861,0.046786,1.251148,1
3,"(20,000 Leagues Under the Sea)",(A Nightmare on Elm Street),0.131148,0.268256,0.081967,0.625000,2.329861,0.046786,1.951316,1
4,(Back to the Future Part II),"(20,000 Leagues Under the Sea)",0.211624,0.131148,0.077496,0.366197,2.792254,0.049742,1.370856,1
5,"(20,000 Leagues Under the Sea)",(Back to the Future Part II),0.131148,0.211624,0.077496,0.590909,2.792254,0.049742,1.927140,1
6,"(20,000 Leagues Under the Sea)",(Batman Returns),0.131148,0.298063,0.073025,0.556818,1.868125,0.033935,1.583859,1
7,(Batman Returns),"(20,000 Leagues Under the Sea)",0.298063,0.131148,0.073025,0.245000,1.868125,0.033935,1.150798,1
8,(Lost in Translation),"(20,000 Leagues Under the Sea)",0.192250,0.131148,0.074516,0.387597,2.955426,0.049302,1.418759,1
9,"(20,000 Leagues Under the Sea)",(Lost in Translation),0.131148,0.192250,0.074516,0.568182,2.955426,0.049302,1.870578,1


In [14]:
rules[ (rules['antecedent_len'] >= 3) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
36210,"(48 Hrs., 5 Card Stud, Terminator 3: Rise of t...",(License to Wed),0.081967,0.301043,0.071535,0.872727,2.899010,0.046859,5.491803,3
36211,"(48 Hrs., 5 Card Stud, License to Wed)",(Terminator 3: Rise of the Machines),0.081967,0.482861,0.071535,0.872727,1.807407,0.031956,4.063232,3
36212,"(48 Hrs., Terminator 3: Rise of the Machines, ...",(5 Card Stud),0.080477,0.298063,0.071535,0.888889,2.982222,0.047548,6.317437,3
36225,"(The 39 Steps, 48 Hrs., License to Wed)",(5 Card Stud),0.076006,0.298063,0.070045,0.921569,3.091863,0.047390,8.949702,3
36227,"(48 Hrs., 5 Card Stud, License to Wed)",(The 39 Steps),0.081967,0.433681,0.070045,0.854545,1.970447,0.034497,3.893443,3
36238,"(48 Hrs., The Million Dollar Hotel, License to...",(5 Card Stud),0.078987,0.298063,0.071535,0.905660,3.038491,0.047992,7.440537,3
36239,"(48 Hrs., The Million Dollar Hotel, 5 Card Stud)",(License to Wed),0.087928,0.301043,0.071535,0.813559,2.702467,0.045065,3.748950,3
36240,"(48 Hrs., 5 Card Stud, License to Wed)",(The Million Dollar Hotel),0.081967,0.463487,0.071535,0.872727,1.882958,0.033544,4.215457,3
36252,"(The 39 Steps, 48 Hrs., 5 Card Stud)",(Rope),0.093890,0.283159,0.071535,0.761905,2.690727,0.044949,3.010730,3
36255,"(48 Hrs., 5 Card Stud, Rope)",(The 39 Steps),0.081967,0.433681,0.071535,0.872727,2.012371,0.035987,4.449649,3


In [15]:
rules[rules['antecedents'] == {'Terminator 3: Rise of the Machines'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
17,(Terminator 3: Rise of the Machines),"(20,000 Leagues Under the Sea)",0.482861,0.131148,0.096870,0.200617,1.529707,0.033544,1.086904,1
36,(Terminator 3: Rise of the Machines),(2001: A Space Odyssey),0.482861,0.129657,0.081967,0.169753,1.309245,0.019361,1.048294,1
149,(Terminator 3: Rise of the Machines),(48 Hrs.),0.482861,0.298063,0.229508,0.475309,1.594660,0.085585,1.337810,1
273,(Terminator 3: Rise of the Machines),(5 Card Stud),0.482861,0.298063,0.192250,0.398148,1.335787,0.048327,1.166296,1
317,(Terminator 3: Rise of the Machines),(A Brief History of Time),0.482861,0.098361,0.076006,0.157407,1.600309,0.028511,1.070077,1
356,(Terminator 3: Rise of the Machines),(A Clockwork Orange),0.482861,0.152012,0.119225,0.246914,1.624304,0.045824,1.126017,1
487,(Terminator 3: Rise of the Machines),(A Nightmare on Elm Street),0.482861,0.268256,0.208644,0.432099,1.610768,0.079113,1.288505,1
581,(Terminator 3: Rise of the Machines),(A River Runs Through It),0.482861,0.196721,0.157973,0.327160,1.663066,0.062984,1.193864,1
635,(Terminator 3: Rise of the Machines),(Aliens vs Predator: Requiem),0.482861,0.134128,0.102832,0.212963,1.587757,0.038066,1.100167,1
694,(Terminator 3: Rise of the Machines),(And Then There Were None),0.482861,0.193741,0.120715,0.250000,1.290385,0.027165,1.075012,1


In [16]:
rules[rules['consequents'] == {'Terminator 3: Rise of the Machines'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
16,"(20,000 Leagues Under the Sea)",(Terminator 3: Rise of the Machines),0.131148,0.482861,0.096870,0.738636,1.529707,0.033544,1.978617,1
37,(2001: A Space Odyssey),(Terminator 3: Rise of the Machines),0.129657,0.482861,0.081967,0.632184,1.309245,0.019361,1.405971,1
148,(48 Hrs.),(Terminator 3: Rise of the Machines),0.298063,0.482861,0.229508,0.770000,1.594660,0.085585,2.248429,1
272,(5 Card Stud),(Terminator 3: Rise of the Machines),0.298063,0.482861,0.192250,0.645000,1.335787,0.048327,1.456728,1
316,(A Brief History of Time),(Terminator 3: Rise of the Machines),0.098361,0.482861,0.076006,0.772727,1.600309,0.028511,2.275410,1
357,(A Clockwork Orange),(Terminator 3: Rise of the Machines),0.152012,0.482861,0.119225,0.784314,1.624304,0.045824,2.397643,1
486,(A Nightmare on Elm Street),(Terminator 3: Rise of the Machines),0.268256,0.482861,0.208644,0.777778,1.610768,0.079113,2.327124,1
580,(A River Runs Through It),(Terminator 3: Rise of the Machines),0.196721,0.482861,0.157973,0.803030,1.663066,0.062984,2.625473,1
634,(Aliens vs Predator: Requiem),(Terminator 3: Rise of the Machines),0.134128,0.482861,0.102832,0.766667,1.587757,0.038066,2.216308,1
695,(And Then There Were None),(Terminator 3: Rise of the Machines),0.193741,0.482861,0.120715,0.623077,1.290385,0.027165,1.372000,1
